In [1]:
import os
import glob
import cv2
import random
import pandas as pd
from skimage import io
import numpy as np
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import h5py

# Network building stuff
import torch
import torch.nn as nn
import torch.nn.functional as F

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
import torchmetrics

In [2]:
import sys
sys.path.append("/home2/sdokania/all_projects/project-noisypixel/")

In [3]:
from src.models import *
from src.dataset.dataloader import OccupancyNetDatasetHDF
from src.trainer import ONetLit
from src.utils import Config, count_parameters

In [6]:
config = Config()

In [7]:
onet = ONetLit(config)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /home2/sdokania/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
# onet

In [17]:
f = open("/ssd_scratch/cvit/sdokania/Shapenet3D/ShapeNet/02691156/test.lst", "r")
data = ["{}_{}.hdf5".format(1234, ix) for ix in f.read().split()[:10]]
f.close()
print(data)

['1234_d18592d9615b01bbbc0909d98a1ff2b4.hdf5', '1234_d18f2aeae4146464bd46d022fd7d80aa.hdf5', '1234_d199612c22fe9313f4fb6842b3610149.hdf5', '1234_d1a887a47991d1b3bc0909d98a1ff2b4.hdf5', '1234_d1a8e79eebf4a0b1579c3d4943e463ef.hdf5', '1234_d1b1c13fdec4d69ccfd264a25791a5e1.hdf5', '1234_d1b28579fde95f19e1873a3963e0d14.hdf5', '1234_d1b407350e61150942d79310bc7e47b3.hdf5', '1234_d1cdd239dcbfd018bbf3143b1cb6076a.hdf5', '1234_d1d308692cb4b6059a6e43b878d5b335.hdf5']


In [20]:
print("\n".join(data))

1234_d18592d9615b01bbbc0909d98a1ff2b4.hdf5
1234_d18f2aeae4146464bd46d022fd7d80aa.hdf5
1234_d199612c22fe9313f4fb6842b3610149.hdf5
1234_d1a887a47991d1b3bc0909d98a1ff2b4.hdf5
1234_d1a8e79eebf4a0b1579c3d4943e463ef.hdf5
1234_d1b1c13fdec4d69ccfd264a25791a5e1.hdf5
1234_d1b28579fde95f19e1873a3963e0d14.hdf5
1234_d1b407350e61150942d79310bc7e47b3.hdf5
1234_d1cdd239dcbfd018bbf3143b1cb6076a.hdf5
1234_d1d308692cb4b6059a6e43b878d5b335.hdf5
